In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split


In [ ]:
# Dataset Preparation
print ("Read Dataset ... ")
train =pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv",parse_dates=['date'],  index_col="date")

test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv",parse_dates=['date'],  index_col="date")


In [ ]:
train.head()

In [ ]:
train['day_of_month'] = train.index.day
train['day_of_week'] = train.index.dayofweek
train['month'] = train.index.month
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
train['country'] = LE.fit_transform(train['country'])
train['store'] = LE.fit_transform(train['store'])
train['product'] = LE.fit_transform(train['product'])


In [ ]:
train

In [ ]:
x=train[['country','store','product','month','day_of_month','day_of_week']]
y=train['num_sold']

In [ ]:
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 

In [ ]:
xgbr.fit(xtrain, ytrain)


In [ ]:
score = xgbr.score(xtrain, ytrain) 
score

In [ ]:
test['day_of_month'] = test.index.day
test['day_of_week'] = test.index.dayofweek
test['month'] = test.index.month

In [ ]:
test['country'] = LE.fit_transform(test['country'])
test['store'] = LE.fit_transform(test['store'])
test['product'] = LE.fit_transform(test['product'])
X_test=test[['country','store','product','month','day_of_month','day_of_week']]
ypred = xgbr.predict(X_test)

In [ ]:
test['num_sold']=np.round(ypred,decimals=0).astype('int')

In [ ]:
test

In [ ]:
print ("Generate Submission File ... ")
test[['row_id','num_sold']].to_csv("submission.csv",index=False)